<a href="https://colab.research.google.com/github/meyresearch/ANI-Peptides/blob/main/DEMO_ANI2x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runtime Environment Setup

Before you start, ensure you have been allocated a GPU runtime. Go to `Runtime -> Change runtime type -> GPU`

Allow kernel to restart after running this cell.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Ensure good conda/mamba installation

In [ ]:
import condacolab
condacolab.check()

Install Dependencies - This takes a little while

*   **OpenMM**

 Base molecular simulation toolkit

*   **TorchANI**

 Implementation of ANI

*   **PyTorch**

 TorchANI runs on the PyTorch machine learning framework 

*   **OpenMM-Torch**

 A plugin for OpenMM that allows PyTorch static computation graphs (TorchANI) to be used in OpenMM as a TorchForce object, an OpenMM Force class 

*   **OpenMM-ML**

 Implements TorchANI as an OpenMM TorchForce using OpenMM-Torch. The glue that brings everything together!

*   **NGLview**

 3D molecule viewer

*   **PyTraj**

 Trajectory library

In [ ]:
# seems to be some bug in pytorch, need to try to find a combination of dependencies that works.

# !mamba install -q openmm openmm-torch pytorch torchani nglview
!mamba install -q -c conda-forge/label/openmm_rc -c conda-forge openmm-torch pytorch=1.8 torchani
!pip install -q pytraj
!git clone https://github.com/openmm/openmm-ml
!pip install openmm-ml/.

Test OpenMM

In [ ]:
!python -m openmm.testInstallation

Fetch sample peptide

In [4]:
!wget -q https://github.com/meyresearch/ANI-Peptides/raw/main/demos/aaa_equilibrated.pdb

# ANI Simulation

ANI2x cannot simulate Na or Cl in solvent. We create a mixed system where our protein is simulated by ANI, and the solvent by AMBER.

In [5]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmmml import MLPotential
from sys import stdout

In [ ]:
# Load sample peptide
pdb = PDBFile('aaa_equilibrated.pdb')

# Create AMBER forcefield
forcefield = ForceField(
    'amber14-all.xml',
    'amber14/tip3pfb.xml'
)

# Create AMBER system
mm_system = forcefield.createSystem(
    pdb.topology, 
    nonbondedMethod=PME,
    nonbondedCutoff=1*nanometer,
    constraints=HBonds
)

# Select protein atoms to be simulated by ANI2x
ml_atoms  = [atom.index for atom in topology.atoms() if atom.residue.name == "ALA"]

# Create ANI2x potential
potential = MLPotential('ani2x')

# Create mixed ANI/AMBER system
ml_system = potential.createMixedSystem(topology, mm_system, ml_atoms)

# Create constant temp integrator
integrator = LangevinMiddleIntegrator(
    300*kelvin,
    1/picosecond,
    0.004*picoseconds
)

# Create simulation and set initial positions
simulation = Simulation(
    pdb.topology,
    ml_system,
    integrator
)

simulation.context.setPositions(pdb.positions)

# Local energy minimisation before ANI
simulation.minimizeEnergy()

# Create reporters for output
simulation.reporters.append(
    PDBReporter('output.pdb', 1000)
)
simulation.reporters.append(
    StateDataReporter(
        stdout, 
        1000, 
        step=True,
        potentialEnergy=True, 
        temperature=True
    )
)

In [ ]:
# Run the simulation through 10000 time steps
simulation.step(10000)